# Comandos PostGIS útiles

## Y distintas formas de acceder desde python (pandas, geopandas, 

In [10]:
def modo1(conn,sql):
    print("Ejecutando "+sql)
    try:
        cur = conn.cursor()
        cur.execute(sql)
        #while cur.description is not None:
        print(cur.statusmessage)
        result = cur.fetchall()
        conn.close()
        sys.stdout.write('Accesed database environment successfully.\n')
        return result
    except psycopg2.Error:
        raise SystemExit(
            'Failed to setup Postgres environment.\n{0}'.format(sys.exc_info())
        )
    return None

def modo2(conn,sql):
    result = gpd.GeoDataFrame.from_postgis(sql, conn, geom_col='trayectos', crs  ={'init': 'epsg:4326'} )#.to_crs(epsg=3857)
    conn.close()
    return result

In [11]:
#set up psycopg2 environment
import sys, os
import psycopg2
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [12]:
querys=[
    "SELECT * FROM pg_catalog.pg_tables WHERE schemaname != 'pg_catalog' AND schemaname != 'information_schema';", #List tablas
    "SELECT postgis_version();",
    "SELECT * from car;",
    "SELECT idtrip,ST_Length(tripline),distance from trip where starttime > timestamp '2019-09-11' ;",
    "DROP table candidatos2;",
    
    
    "SELECT driver_iddriver as conductor, avg(distance) as promedio  \
    FROM Trip where starttime > timestamp '2019-07-01' group by driver_iddriver;",
    
    "SELECT idtrip,ST_Length((tripline))*100,distance from trip where distance <> 0 AND (ST_Length((tripline))*100-distance)/distance < 0.05;",
    
    "CREATE TABLE candidatos2 AS SELECT idtrip,distance,\
     ST_Intersection(tripline, ST_MakeEnvelope(-4.512359,36.703857, -4.455261,36.736785))::geometry as trayectos \
     FROM simplified_trip WHERE  \
     ST_Intersects(tripline, ST_MakeEnvelope(-4.512359,36.703857 , -4.455261,36.736785, 4326)::geography);",

     "CREATE TABLE candidatos AS SELECT idtrip,distance,\
     ST_LineMerge(ST_Intersection(tripline, ST_MakeEnvelope(-4.512359,36.703857, -4.455261,36.736785))::geometry) as trayectos \
     FROM simplified_trip WHERE  \
     ST_Intersects(tripline, ST_MakeEnvelope(-4.512359,36.703857 , -4.455261,36.736785, 4326)::geography) \
     AND  distance <> 0 AND (ST_Length((tripline))*100-distance)/distance < 0.05;",

    "select idtrip,st_npoints(tripline),st_numgeometries(tripline), (select count(*) \
    from tripsample where trip_idtrip = idtrip) as nodos  from trip  where starttime > timestamp '2019-10-1' order by idtrip;",
    
    "SELECT min(trip_idtrip) as idtrip, max(nsamples) as sample_cnt, \
    min(starttime) as start_time, min(vehspeed) as minspeed, max(vehspeed) as maxspeed, \
    max(toffset) as tot_time, sum(dspace) as tot_space, sum(dfuel) as tot_fuel_l, sum(dfuel)/sum(dspace) as fuel_lpermeter,\
    date_part('dow',MIN(starttime)) + date_part('hour',MIN(starttime)) / 24.0 as weekoffset_days, \
	date_part('dow',MIN(starttime)) as day_of_week, date_part('hour',MIN(starttime)) as hour_of_day \
    FROM tripsamples_path  WHERE time > TIMESTAMP '2018-07-01' GROUP BY trip_idtrip;"
    
    

]    

In [13]:
conn = psycopg2.connect("dbname = 'umaobd' user = 'obd_readpriv' host = 'atomic3.dmz.ac.uma.es' password = 'vc0910$$'")
result=modo1(conn,"select * from candidatos;")
result[0]

Ejecutando select * from candidatos;
SELECT 176
Accesed database environment successfully.


(588,
 18.8397521972656,
 '010200000020000000B89388F02FD211C0494A7A185A5B42407C0DC17119D711C01D90847D3B5B4240EB89AE0B3FD811C0137EA99F375B42401BB7989F1BDA11C08AAE0B3F385B42408DEF8B4B55EA11C0FFEA71DF6A5B42403657CD7344EE11C02E3BC43F6C5B4240300E2E1D73EE11C0CDCAF6216F5B4240BEA085048CEE11C0834E081D745B42405D4D9EB29AEE11C06EC1525DC05B42403A1E335019EF11C09C685721E55B4240FC1BB4571FEF11C024ED461FF35B4240EFC3414294EF11C041EF8D21005C4240A4DDE8633EF011C08369183E225C42403EB0E3BF40F011C0F1D8CF62295C4240774CDD955DF011C07DC9C6832D5C424005FC1A4982F011C0F4F928232E5C4240541D7233DCF011C01C08C902265C424089B0E1E995F211C0B6D4415E0F5C42400585419946F311C0ACC266800B5C42401361C3D32BF511C0592E1B9DF35B4240130F289B72F511C0B08C0DDDEC5B4240FB8EE1B19FF511C01FF64201DB5B4240574277499CF511C0382F4E7CB55B4240EACBD24ECDF511C0B4CBB73EAC5B424029779FE3A3F511C029779FE3A35B4240685C381092F511C0CD55F31C915B42404528B682A6F511C0C022BF7E885B424067EDB60BCDF511C06E30D461855B424067B8019F1FF611C0527FBDC2825B4240F44C2F3196F911C0CFA0A17F825

In [14]:
conn = psycopg2.connect("dbname = 'umaobd' user = 'obd_readpriv' host = 'atomic3.dmz.ac.uma.es' password = 'vc0910$$'")
result=modo2(conn,"select * from candidatos")
result.head()

,idtrip,distance,trayectos
0,588,18.839752,"LINESTRING (-4.455261 36.713687, -4.460058 36...."
1,74,6.847943,"LINESTRING (-4.477214 36.715649, -4.4770949999..."
2,34,109.317841,"LINESTRING (-4.512359 36.711619, -4.507536 36...."
3,37,7.124002,"(LINESTRING (-4.457788 36.713185, -4.457853 36..."
4,10,0.065981,"LINESTRING (-4.472852 36.71299399999999, -4.47..."
